In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import (Input, Layer, Dense, AveragePooling2D, UpSampling2D)

import tensorflow_datasets as tfds

from tensorflow.keras.applications.inception_v3 import InceptionV3

import numpy as np
import pandas as pd

from datetime import datetime
import os
import sys
import pickle
import time
import argparse
from tqdm import tqdm
import gc

sys.path.insert(0, '/vast/nj594/xai/helpers')
from evaluate import evaluate

# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

In [2]:
# Load Data

data_dir = os.path.join('..', 'data')

X_val = np.load(os.path.join(data_dir, 'X_val_processed.npy'), allow_pickle=True)
X_test = np.load(os.path.join(data_dir, 'X_test_processed.npy'), allow_pickle=True)

y_val = np.load(os.path.join(data_dir, 'y_val.npy'), allow_pickle=True)
y_test = np.load(os.path.join(data_dir, 'y_test.npy'), allow_pickle=True)
y_test_fixed = np.load(os.path.join(data_dir, 'y_test_fixed.npy'), allow_pickle=True)

preds = np.load(os.path.join(data_dir, 'predictions.npy'), allow_pickle=True)
preds_discrete = np.eye(5)[preds.argmax(1)]
preds_sample  = np.load(os.path.join(data_dir, 'predictions_sampled.npy'), allow_pickle=True)

preds_val = np.load(os.path.join(data_dir, 'predictions_val.npy'), allow_pickle=True)
preds_discrete_val = np.eye(5)[preds_val.argmax(1)]

In [3]:
# Sample a Label from the Model Preds
# cum_prob = np.cumsum(preds, axis=-1) # shape (n1, n2, m)
# r = np.random.uniform(size=(preds.shape[0], 1))

# preds_sample = np.argmax(cum_prob > r, axis=-1)
# preds_sample = np.eye(5)[preds_sample]

# np.save(os.path.join(data_dir, 'predictions_sampled.npy'), preds_sample)

In [4]:
# Load Evaluator Model
print('Loading Evaluator')

eval_dir = 'evaluator-data'
evaluator_model = tf.keras.models.load_model(os.path.join(eval_dir, 'surrogate.h5'))

OPTIMIZER = tf.keras.optimizers.Adam(1e-3)
METRICS = [ 
  tf.keras.metrics.AUC(name='auroc'),
  tf.keras.metrics.AUC(curve='PR', name='auprc'),
  tf.keras.metrics.TopKCategoricalAccuracy(k=1, name='accuracy'),
]

evaluator_model.compile(
    loss='categorical_crossentropy',
    optimizer=OPTIMIZER,
    metrics=METRICS,
)

gc.collect()

Loading Evaluator


2022-05-16 23:10:03.108678: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-05-16 23:10:03.109722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 32252 MB memory:  -> device: 0, name: Vega 20, pci bus id: 0000:8a:00.0


16588

In [ ]:
methods = ['fastshap-dkl', 'kernelshap_s-dkl', 
           'fastshap', 'kernelshap', 'smoothgrad', 'gradcam', 'lime', 
           'integratedgradients', 'kernelshap_s', 'realx']
results = {}
for method in methods:
    print(method)
    results[method] = {} 
    ## Load Explanations
    if method  not in ['smoothgrad', 'integratedgradients', 'realx', 'realx-data', 'gradcam']:
        exp_file_retro = os.path.join('..', method, 'shap_values-retrospective.pkl')
        exp_file_pro = os.path.join('..', method, 'shap_values-prospective.pkl')
    elif method == 'gradcam':
        exp_file_retro = os.path.join('..', method, 'explanations.pkl')
        exp_file_pro = exp_file_retro
    else:
        exp_file_retro = os.path.join('..', method, 'explanations-retrospective.pkl')
        exp_file_pro = os.path.join('..', method, 'explanations-prospective.pkl')
        
    with open(exp_file_retro, 'rb') as f:
        explanations_retro = pickle.load(f)
    with open(exp_file_pro, 'rb') as f:
        explanations_pro = pickle.load(f)
        
    if 'realx' in method:
        print(method)
        explanations_retro += np.random.random(explanations_retro.shape) * 1e-7
        explanations_pro += np.random.random(explanations_pro.shape) * 1e-7
        
        
    ## Evaluate Explanations
    with open('results_fixed-{}.pkl'.format(method), 'rb') as f:
        result_temp = pickle.load(f)
    
    ### Retrospective Evaluation

    # Exclusion
    results[method]['retro_ex'] = result_temp['retro_ex']
    # Inclusion
    results[method]['retro_in'] = result_temp['retro_in']
    
    ### Prospective Evaluation 

    # Exclusion
    results[method]['pro_ex'] = evaluate(X_test, explanations_pro, evaluator_model, preds_sample, y_test_fixed, 
                                         mode = 'exclude', method = method)
    # Inclusion
    results[method]['pro_in'] = evaluate(X_test, explanations_pro, evaluator_model, preds_sample, y_test_fixed, 
                                         mode = 'include', method = method) 
    
    print(results[method])
    
    with open('results_sampled-{}.pkl'.format(method), 'wb') as f:
        pickle.dump(results[method], f)

fastshap-dkl
100


2022-05-16 23:15:37.034216: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-05-16 23:15:37.053916: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-16 23:15:37.058116: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-16 23:15:37.060542: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-16 23:15:38.310747: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIOpen(HIP): Error [FindRecordUnsafe] Ill-formed record: key not found: /home/nj594/.config/miopen//gfx906_60.HIP.2_11_0_.ufdb.txt#486
MIO

99


2022-05-16 23:16:38.950367: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-16 23:16:38.954438: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-16 23:16:38.956751: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


95


...

25


2022-05-17 01:31:48.041479: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-17 01:31:48.045407: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-17 01:31:48.047765: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


15


2022-05-17 01:32:39.541463: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-17 01:32:39.546035: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.
2022-05-17 01:32:39.548353: I tensorflow/core/common_runtime/gpu_fusion_pass.cc:507] ROCm Fusion is enabled.


10


In [ ]:
results = {}
for method in methods:
    with open('results_sampled-{}.pkl'.format(method), 'rb') as f:
        results[method] = pickle.load(f)

In [ ]:
with open('results_sampled.pkl', 'wb') as f:
    pickle.dump(results, f)